# Problem 3 and Challenge Problem

1. Find out the best data-structure to represent / store the graph in memory.
2. Devise an algorithm to assign the labels to the vertices using vertex **k**-labeling definition. (**Main Task**)
3. How traversing will be applied?
4. Store the labels of vertices and weights of the edges as an outcome.
5. Compare your results with mathematical property and tabulate the outcomes for comparison.


Answer to questions and rough outline.

1. Edges can be represented in an array of lists or 2D-Array. For the current problem, I believe an array of lists would be best. It can be implemented as a dictionary with the key referencing a list of adjacent vectors.
2. WIP
3. Simple BFS and DFS Functions have been defined (will be improved up on once vertex k-labeling is implemented)
4. This *may* be acomplished with the __str__ function in the Edge class.
5. WIP

__k-labeling__: Each vertex is labeled with a unique positive integer, and the absolute difference between the labels of any two adjacent vertices is at least \(k\).

In [ ]:
class Edge:
  # Data Fields
  source = None
  dest = None
  weight = None

  # Constructor
  def __init__(self, source, dest, weight=1.0):
    self.source = source
    self.dest = dest
    self.weight = weight

  # Destructor
  def __del__(self):
    self.source = None
    self.dest = None
    self.weight = None

  # Member Functions
  # Compare edges for equality
  def __eq__(self, other):
    # Check if 'other' is an instance of the same class
    if not isinstance(other, Edge):
      return False
    return self.source == other.source and self.dest == other.dest

  # Returns the destination vertex
  def get_dest(self):
    return self.dest

  # Returns te source vertex
  def get_source(self):
    return self.source

  # Returns the weight
  def get_weight(self):
    return self.weight

  # Returns a string representation of the edge
  def __str__(self):
    return f"({self.source}, {self.dest}, {self.weight})"

In [ ]:
from collections import deque

class Graph:
  # Data fields
  directed = False
  num_v = 0
  graph = {}
  edges = {}

  # Constructor
  def __init__(self, num_v=0, directed=False):
    self.directed = directed
    self.num_v = num_v

  # Destructor
  def __del__(self):
    self.directed = None
    self.num_v = None
    self.graph = None
    self.edges = None

  # Member functions
  # Returns the number of vertices
  def get_num_v(self):
    return self.num_v

  # Returns true if the graph is directed
  def is_directed(self):
    return self.directed

  # Returns the list of vertices adjacent to a given source
  def begin(self, source):
    return self.graph[source]

  # Gets the edge between 2 vertices
  def get_edge(self, source, dest):
    if self.is_edge(source, dest):
      for edge in self.edges[source]:
        if edge.get_dest() == dest:
          return edge
    return None

  # Inesrts a new edge into the "edges" dictionary
  def insert_edge(self, edge):
    src, dest, weight = edge.get_source(), edge.get_dest(), edge.get_weight()
    # Base case: Exit function if the edge already exists
    if self.is_edge(src, dest):
      return

    # Inserts the edge into the "edges" dictionary
    if src in self.edges:
      self.edges[src].append(edge)
    else:
      self.edges[src] = [edge]

    # Inserts the edge with the source and dest swapped if the graph is not directed
    if not self.directed:
      opp_edge = Edge(dest, src, weight)
      self.insert_edge(opp_edge)

    # Insert the vertices into the "graph" dictionary
    self.insert_graph(edge)
    if not self.directed:
      opp_edge = Edge(dest, src, weight)
      self.insert_graph(opp_edge)

  # Inserts the vertices from 'edge' into the "graph" dictionary
  def insert_graph(self, edge):
    src, dest, weight = edge.get_source(), edge.get_dest(), edge.get_weight()
    # Inserts the vertices ordered by smallest weight to largest weight
    if src in self.graph:
      # Prevents duplicate insertions
      if dest in self.graph[src]:
        return
      for i in self.graph[src]:
        e = self.get_edge(src, i)
        if e.get_weight() > weight:
          index = self.graph[src].index(i)
          self.graph[src].insert(index, dest)
          return    # ensures no append at the end of list

      # Only reached if no insertion happened earlier
      self.graph[src].append(dest)
    else:
      self.graph[src] = [dest]

  # Determines whether an edge exists from source to dest
  def is_edge(self, source, dest):
    if source in self.edges:
      for i in self.edges[source]:
        if i.get_dest() == dest:
          return True
    return False

  # Simple BFS (change to account for weight and source labels)
  def bfs(self, start):
    seen = set([start])
    q = deque([start])
    order = []
    while q:
        v = q.popleft()
        order.append(v)
        for nbr in self.graph[v]:
            if nbr not in seen:
                seen.add(nbr)
                q.append(nbr)
    return order

  # Simple DFS (change to account for weight and source labels)
  def dfs(self, start):
    seen = set()
    order = []
    def rec(v):
        seen.add(v)
        order.append(v)
        for nbr in self.graph[v]:
            if nbr not in seen:
                rec(nbr)
    rec(start)
    return order

    # Create graph with k-labeling
    def create_k_graph(self):
      return 0

In [ ]:
def main():
  G = Graph(6)
  edges = []

  edges.append(Edge(0, 1))
  edges.append(Edge(0, 2))
  edges.append(Edge(1, 3))
  edges.append(Edge(1, 4))
  edges.append(Edge(2, 4))
  edges.append(Edge(3, 5))
  edges.append(Edge(4, 5))

  for e in edges:
    G.insert_edge(e)

  print("BFS: ", G.bfs(0))
  print("DFS: ", G.dfs(0))

  for i in range(G.get_num_v()):
    print(str(i), ": ", G.begin(i))

if __name__ == "__main__":
  main()

BFS:  [0, 1, 2, 3, 4, 5]
DFS:  [0, 1, 3, 5, 4, 2]
0 :  [1, 2]
1 :  [0, 3, 4]
2 :  [0, 4]
3 :  [1, 5]
4 :  [1, 2, 5]
5 :  [3, 4]
